In [1]:
import os
import faiss
import pickle
import numpy as np

In [2]:
features_path = r'C:\Retrieval System\features\features_clip'

In [3]:
def normalize_data(data):
    # Kiểm tra nếu data là một vector 1 chiều
    if data.ndim == 1:
        normalized_data = data / np.linalg.norm(data)
    else:
        # Chuẩn hóa dữ liệu nếu là mảng 2 chiều
        normalized_data = data / np.linalg.norm(data, axis=1, keepdims=True)
    
    return normalized_data

In [4]:
save_name_file = []
features = []

for file in os.listdir(features_path):
    file_path = os.path.join(features_path, file)

    with open(file_path, "rb") as f:
        image_features_data = pickle.load(f) # 1 list, các phần tử là dict với image_name và features

        for item in image_features_data:
            features.append(item['features'])
            save_name_file.append(item['image_name'])

db = np.array(features)
print(db.shape)

d = db.shape[1]
print(d)
nb = db.shape[0]
# nq = query_beit.shape[0]

index = faiss.IndexFlatIP(d)   # build the index
print(index.is_trained)
index.add(normalize_data(db))  # add vectors to the index
print(index.ntotal)

# save_path = os.path.join(r'C:\Retrieval System\Multimedia-Information-Retrieval\index', 'feature_paris_clip.pkl')
# Lưu index và danh sách vào file
# with open(save_path, "wb") as f:
#     pickle.dump({"index": index, "map_index": save_name_file}, f)
# print('hehe')

(6392, 512)
512
True
6392


In [5]:
from PIL import Image
import torch
from torchvision.transforms import Compose, Resize, Normalize, ToTensor
from transformers import CLIPProcessor, CLIPModel
import numpy as np

gt_path = r'C:\Retrieval System\data\paris_120310_gt'
data_path = r'C:\Retrieval System\data\paris'
features_path = r'C:\Retrieval System\features\features_paris_clip_3'

model = CLIPModel.from_pretrained('openai/clip-vit-base-patch32')
processor = CLIPProcessor.from_pretrained('openai/clip-vit-base-patch32')

model.eval()

transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
])

def process_queries(query_file, device='cpu'):
    """
    Đọc file query, cắt ảnh theo bounding box và extract feature theo từng bb.
    
    :param query_file: Đường dẫn đến file query.txt
    :param image_folder: Thư mục chứa ảnh
    """
    with open(query_file, 'r') as file:
        features = []
        for line in file:
            # Tách dòng query thành các phần tử
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            
            image_name = parts[0] + '.jpg'  # Tên file ảnh
            x1, y1, x2, y2 = map(float, parts[1:])  # Bounding box
            
            # Đường dẫn đầy đủ đến ảnh
            image_path = os.path.join(data_path, image_name.split('_')[1], image_name)
            
            # Kiểm tra ảnh tồn tại
            if not os.path.exists(image_path):
                print(f"Ảnh {image_name} không tồn tại trong thư mục {os.path.join(data_path, image_name.split('_')[1])}.")
                continue
            
            # Mở ảnh và cắt theo bounding box
            with Image.open(image_path).convert('RGB') as img:
                cropped_img = img.crop((x1, y1, x2, y2))  # Cắt ảnh
                # Extract feature của ảnh crop
                cropped_img = transform(cropped_img).unsqueeze(0).to(device)
                with torch.no_grad():
                    feature = model.get_image_features(cropped_img)
                feature = feature.squeeze().cpu().numpy()
                features.append(feature)
        
        return features

query = []
map_qr = []
for file in os.listdir(gt_path):
    if 'query' in file:
        # print(file)
        map_qr.append(file.split('.')[0])
        query_file = os.path.join(gt_path, file)
        features = process_queries(query_file=query_file)
        if len(features) == 1:
            query.append(features[0])

query = np.array(query)
map_qr = [name[:-6] if name.endswith("_query") else name for name in map_qr]
print(map_qr)
query.shape

c:\Users\DELL\miniconda3\envs\CS336\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\DELL\miniconda3\envs\CS336\Lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\DELL\miniconda3\envs\CS336\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


['defense_1', 'defense_2', 'defense_3', 'defense_4', 'defense_5', 'eiffel_1', 'eiffel_2', 'eiffel_3', 'eiffel_4', 'eiffel_5', 'invalides_1', 'invalides_2', 'invalides_3', 'invalides_4', 'invalides_5', 'louvre_1', 'louvre_2', 'louvre_3', 'louvre_4', 'louvre_5', 'moulinrouge_1', 'moulinrouge_2', 'moulinrouge_3', 'moulinrouge_4', 'moulinrouge_5', 'museedorsay_1', 'museedorsay_2', 'museedorsay_3', 'museedorsay_4', 'museedorsay_5', 'notredame_1', 'notredame_2', 'notredame_3', 'notredame_4', 'notredame_5', 'pantheon_1', 'pantheon_2', 'pantheon_3', 'pantheon_4', 'pantheon_5', 'pompidou_1', 'pompidou_2', 'pompidou_3', 'pompidou_4', 'pompidou_5', 'sacrecoeur_1', 'sacrecoeur_2', 'sacrecoeur_3', 'sacrecoeur_4', 'sacrecoeur_5', 'triomphe_1', 'triomphe_2', 'triomphe_3', 'triomphe_4', 'triomphe_5']


(55, 512)

In [6]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

k = 5
distances, indices = index.search(query, k)
print(indices)

os.environ["KMP_DUPLICATE_LIB_OK"] = "FALSE"

[[ 329   43  162   42  316]
 [ 164   43  329  359  298]
 [  86  125  329  162   43]
 [ 175  224  164   43  178]
 [  43  329  162   86  132]
 [1255 1411 1748 1923  626]
 [1274  686  392  785  477]
 [1411 1224  420 1748  838]
 [1923 1748  626 1255 1228]
 [ 922  657  460 1360  477]
 [2182 2294 2035 2455 2289]
 [2182 2035 2294 2447 2225]
 [2182 2263 2035 2060 2468]
 [2294 2434 2188 2225 2147]
 [2035 2182 2294 2263 2468]
 [2647 2579 1068 2654 2527]
 [2582 2655 2708 2606 2511]
 [2646 2645 2608 2708 2610]
 [2608 2645 2646 2708 2582]
 [2582 2567 2708 2579 1068]
 [2859 3158 2993 3082 3000]
 [3082 3108 3158  757 3000]
 [3082 2901 2993 2859  757]
 [3108 2822 3082 2993 2859]
 [2822 3108  757 3158 3082]
 [3311 3876 3571 3731 3240]
 [3240 3731 3453 3311 3876]
 [3240 3876 3731 3571 3311]
 [3876 3311 3571 3731 3240]
 [3731 3571 3876 3453 3240]
 [3966 4352 4250 4240 4135]
 [4123 4251 4228 4135 4076]
 [4238 4076 4135 4163 4284]
 [4135  985 4250 4240 4284]
 [4179 4135 4078 4163 4284]
 [4813 4427 4769 468